In [ ]:
%env CUDA_VISIBLE_DEVICES=5

In [ ]:
import numpy as np
import cupy as cp

In [ ]:
xp = np

X = 1500
Y = 1500
Z = 20
K = 3
data = np.random.rand(X,Y,Z).astype('float32')

# numpy

In [ ]:
def vectors_comparison(vector1, vector2):
    res = xp.multiply(vector1, vector2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def vectors_comparison1(matrix1, matrix2):
    return (xp.power(matrix1 * matrix2, 2)).sum(axis=2)

def compare_matrices(a_pad, data, xk, yk, K, comp_function):
    comparison = comp_function(data, a_pad[xk:xk+data.shape[0], K+yk:K+yk+data.shape[1], :])
    res = xp.pad(comparison, ((xk, 0), (K, K)), constant_values=xp.nan)[0:data.shape[0], K-yk:K-yk+data.shape[1]]
    return comparison, res

def get_comparison(data, X, Y, Z, K, comp_function):
    K = K + 1
    a_pad = xp.pad(data, ((0, K), (K, K), (0, 0)), constant_values=xp.nan)
    result = []    
    for xk in range(0, K):
        for yk in range(0, K):
            if (xk!=0) or (yk!=0):   
                result.extend(compare_matrices(a_pad, data, xk, yk, K, comp_function))
                if (yk!=0) and (xk!=0):
                    result.extend(compare_matrices(a_pad, data, xk, -yk, K, comp_function))
    cp.cuda.stream.get_current_stream().synchronize()
    return result

In [ ]:
# data_n = cp.asarray(data)
data_n = data

In [ ]:

res = get_comparison(data_n, X, Y, Z, K, vectors_comparison)

In [ ]:
res[0].shape

In [ ]:
def vectors_comparison(vector1, vector2):
    res = xp.multiply(vector1, vector2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def vectors_comparison1(matrix1, matrix2):
    return (xp.power(matrix1 * matrix2, 2)).sum(axis=2)

def compare_matrices(a_pad, data, xk, yk, K, comp_function):
    comparison = comp_function(data, a_pad[xk:xk+data.shape[0], K+yk:K+yk+data.shape[1], :])
    #res = xp.pad(comparison, ((xk, 0), (K, K)), constant_values=xp.nan)[0:data.shape[0], K-yk:K-yk+data.shape[1]]
    res = xp.pad(
        comparison[:data.shape[0]-xk, max(0, -yk):min(data.shape[1], data.shape[1]-yk)], 
        ((xk, 0), (max(0, yk), -min(0, yk))), 
        constant_values=xp.nan
    )
    return comparison, res

def get_comparison2(data, X, Y, Z, K, comp_function):
    K = K + 1
    a_pad = xp.pad(data, ((0, K), (K, K), (0, 0)), constant_values=xp.nan)
    result = []
    for xk in range(0, K):
        for yk in range(0, K):
            if (xk!=0) or (yk!=0):   
                result.extend(compare_matrices(a_pad, data, xk, yk, K, comp_function))
                if (yk!=0) and (xk!=0):
                    result.extend(compare_matrices(a_pad, data, xk, -yk, K, comp_function))
    cp.cuda.stream.get_current_stream().synchronize()
    return result

In [ ]:
res2 = get_comparison2(data_n, X, Y, Z, K, vectors_comparison)

In [ ]:
xp.allclose(res, res2, equal_nan=True)

In [ ]:
def compute_local(self, function, data, bad_traces, kernel_size=3,
                      normalize=True, agg='mean', amortize=False, axis=0, device='cpu', pbar=None):
        i_range, x_range = data.shape[:2]
        k = kernel_size // 2

        # Transfer to GPU, if needed
        data = to_device(data, device)
        bad_traces = to_device(bad_traces, device)
        xp = cp.get_array_module(data) if (cp is not np) else np

        # Compute data statistics
        data_stds = data.std(axis=-1)
        bad_traces[data_stds == 0.0] = 1
        if normalize:
            data_n = data - data.mean(axis=-1, keepdims=True)
        else:
            data_n = data

        # Pad everything
        padded_data = xp.pad(data_n, ((k, k), (k, k), (0, 0)), constant_values=xp.nan)
        padded_stds = xp.pad(data_stds, k, constant_values=0.0)
        padded_bad_traces = xp.pad(bad_traces, k, constant_values=1)

        # Compute metric by shifting arrays
        total = kernel_size * kernel_size - 1
        pbar = Notifier(pbar, total=total) if pbar else None

        accumulator = Accumulator(agg=agg, amortize=amortize, axis=axis, total=total)
        for i in range(kernel_size):
            for j in range(kernel_size):
                if i == j == k:
                    continue

                shifted_data = padded_data[i:i+i_range, j:j+x_range]
                shifted_stds = padded_stds[i:i+i_range, j:j+x_range]
                shifted_bad_traces = padded_bad_traces[i:i+i_range, j:j+x_range]

                computed = function(data_n, shifted_data, data_stds, shifted_stds)
                computed[shifted_bad_traces == 1] = xp.nan
                accumulator.update(computed)
                if pbar:
                    pbar.update()
        if pbar:
            pbar.close()

        result = accumulator.get(final=True)
        return from_device(result)